In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Load CSV data
data = pd.read_csv("daily_data.csv")

print(data.head())

    Sl.No  ItemID                   Item_type  \
0  121959   51682  INJ - NON-OPIOID ANALGESIC   
1  671156    9426  Caps/Tab - HYPO-GLYCAEMICS   
2  131933   53188          CapsTabANTIBIOTICS   
3  365839   24347         Caps/Tab - MINERALS   
4  259179   68822         Caps/Tab - VITAMINS   

                          Item_Name Expiry_date        Issue_Date   IssueDate  \
0  Diclofenac Sodium Inj. 75mg./ml.  31-07-2024  03-05-2024 11:27  03-12-2023   
1             Metformin SRTab. 1gm.  31-08-2025  24-05-2024 17:19  24-11-2022   
2                    Syncefix Caps.  31-07-2024  21-05-2024 17:59  21-05-2022   
3               Triple A Cal. Caps.  28-02-2025  22-01-2024 10:26  22-08-2023   
4           Meconeuron OD 1500 Tab.  31-12-2024  02-04-2024 14:17  02-04-2022   

    Time  IssueQuantity Issue_Department Medicine_Form  
0  11:27              2         Pharmacy          Nos.  
1  17:19             30       SUB STORES          Nos.  
2  17:59             10         PHARMACY       

In [3]:
# Convert Sales_Date to datetime and sort
data['IssueDate'] = pd.to_datetime(data['IssueDate'], errors='coerce', dayfirst=True)
data = data.sort_values(by=['Item_Name', 'IssueDate'])

# Select relevant columns for LSTM training
data = data[['Item_Name', 'IssueDate', 'IssueQuantity']]
data['Item_Name'] = data['Item_Name'].str.strip().str.lower().str.rstrip('.')

In [4]:
# Pivot data to create a time series per medicine
pivot_data = data.groupby(['IssueDate', 'Item_Name'])['IssueQuantity'].sum().unstack(fill_value=0)
pivot_data.columns = [col.strip().lower().rstrip('.') for col in pivot_data.columns]

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(pivot_data)

In [5]:
# Prepare LSTM input sequences
def create_sequences(data, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(data[i + seq_length])
    return np.array(sequences), np.array(targets)

seq_length = 30  # Use past 30 days for prediction
X, y = create_sequences(scaled_data, seq_length)

In [6]:
# Split into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [7]:
# Define LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, X.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(X.shape[2])
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
print(model)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0079 - val_loss: 0.0079
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0072 - val_loss: 0.0075
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0071 - val_loss: 0.0074
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0068 - val_loss: 0.0073
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0068 - val_loss: 0.0074
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0067 - val_loss: 0.0073
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0061 - val_loss: 0.0073
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0062 - val_loss: 0.0072
Epoch 9/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0069 - val_loss: 0.0072
Epoch 10/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0065 - val_loss: 0.0072
Epoch 11/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0063 - val_loss: 0.0072
Epoch 12/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0

In [9]:
# Save trained model
model.save('lstm_stock_prediction.h5')

print("Model training complete. Saved as 'lstm_stock_prediction.h5'.")

Model training complete. Saved as 'lstm_stock_prediction.h5'.
